In [27]:
# gerekli paketlerin import edilmesi
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [29]:
print(cv2.__version__)

4.2.0


In [4]:
def detect_and_predict_mask(frame, faceNet, maskNet):
	# maskenin bulundugu alan icin bir cerceve olusturuyoruz
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 1.0, (224, 224),
		(104.0, 177.0, 123.0))

	# yuzun algilanmasi
	faceNet.setInput(blob)
	detections = faceNet.forward()
	print(detections.shape)

	# yapilacak tahmin icin gerekli olan listelerin olusturulmasi
	faces = []
	locs = []
	preds = []

	# detection icin dongu olusturuyoruz
	for i in range(0, detections.shape[2]):
		# detection icin confidence orani
		confidence = detections[0, 0, i, 2]

		# confidence oranini minimum duzeyden daha fazla olmasi kosulunu koyuyoruz
		# bu sayede zayif olan algilamalari filtrelemis olacagiz
		if confidence > 0.5:
			# maske icin sinirli kutuya ait (x, y) koordinatlarinin hesaplanmasi
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX, startY, endX, endY) = box.astype("int")

			# sinirladigimiz kutunun boyutunun cercevenin boyutlari dahilinde olup olmadigini kontrol ediyoruz
			(startX, startY) = (max(0, startX), max(0, startY))
			(endX, endY) = (min(w - 1, endX), min(h - 1, endY))

			# yuz icin gerekli BGR'den RGB donusturme, 224x224'e boyutlandirma ve onisleme islemlerini gerceklestiriyoruz
			face = frame[startY:endY, startX:endX]
			face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
			face = cv2.resize(face, (224, 224))
			face = img_to_array(face)
			face = preprocess_input(face)

			faces.append(face)
			locs.append((startX, startY, endX, endY))

# 0 dan fazla yuz algilandiginda tahmin uretmesi icin dongu kuruyoruz
	if len(faces) > 0:
		# daha hiz cikarim icin tahminleri tek tek yapmak yerine butun yuzler yuzerinde ayni anda tahmin uretecegiz
		faces = np.array(faces, dtype="float32")
		preds = maskNet.predict(faces, batch_size=32)

	return (locs, preds)


In [26]:
# modelimizi cagiriyoruz
prototxtPath = r"C:\Users\Zehra\BitirmeProjesi2\deploy.prototxt"
weightsPath = r"C:\Users\Zehra\BitirmeProjesi2\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# kendi olusturdugumuz modeli maskNet isimli degiskene yukleyelim
maskNet = load_model("mask_detector.model")

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\dnn\src\caffe\caffe_io.cpp:1121: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "C:\Users\Zehra\BitirmeProjesi2\deploy.prototxt" in function 'cv::dnn::ReadProtoFromTextFile'


In [17]:
# video akisini baslatalim
print("[INFO] starting video stream...")
vs = VideoStream(src=0).start()


[INFO] starting video stream...


In [18]:
# video akisi icin bir while dongusu tanimliyoruz
while True:
	# videodan aldigimiz goruntu cercevesini 400 piksel olacak sekilde bicimlendiriyoruz
	frame = vs.read()
	frame = imutils.resize(frame, width=400)

	# cercevedeki yuzun algilanip maske takili olup olmadiginin tahmin edilmesi
	(locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

	# algilanan yuz konumlari degisiklik gosterecegi icin bir dongu olusturuyoruz
	for (box, pred) in zip(locs, preds):

		(startX, startY, endX, endY) = box
		(mask, withoutMask) = pred

		# videodaki kutu ve metin icin kullanacagimiz sinif etiketini ve rengini belirliyoruz
		label = "Mask" if mask > withoutMask else "No Mask"
		color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

		# metin etiketine olasiligi da ekliyoruz
		label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

		# metin etiketini ve sinirlayici kutunun cikti cercevesinde goruntulenmesi
		cv2.putText(frame, label, (startX, startY - 10),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
		cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

	# ciktinin gosterilmesi (cerceve)
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF

	# space tusuna basildiginda dongulen cikilmasi
	if key == ord(" "):
		break

NameError: name 'faceNet' is not defined

In [ ]:
cv2.destroyAllWindows()
vs.stop()